In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from Webscrape import Webscrape

In [11]:
base_url: str = 'https://www.zorgkaartnederland.nl'

In [12]:
facility: str = 'tandartsenpraktijk'

In [13]:
scrape = Webscrape(base_url=base_url)

In [14]:
results = scrape.get_filter_result(facility=facility)

In [31]:
dict_result = scrape.get_filter_links(results=results)
dict_result

{'Dent-S': 'https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-dent-s-amsterdam-3033585',
 'MondCleanic, Center for Dental Excellence': 'https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-mondcleanic-center-for-dental-excellence-noordwijk-3045043',
 'Tridenzo mondzorg': 'https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-tridenzo-mondzorg-breda-3058852',
 "Tandheelkundig Centrum 't Veld": 'https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-tandheelkundig-centrum-t-veld-t-veld-3040694',
 'Praktijk voor Endodontologie Bussum': 'https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-praktijk-voor-endodontologie-bussum-bussum-3036956',
 'Ittmann Tandartsen': 'https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-ittmann-tandartsen-velp-3043194',
 'TandartsCraanen.nl': 'https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-tandartscraanen-nl-ubbena-3020937',
 'Tandartsenpraktijk Quis

In [30]:
scrape.get_page_total(facility=facility)

232

## Debug

In [65]:
import requests
from bs4 import BeautifulSoup
import json
import re
from typing import Tuple, Optional

In [68]:
test_url = dict_result.get('Ittmann Tandartsen')
test_url

'https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-ittmann-tandartsen-velp-3043194'

In [69]:
response = requests.get(url=test_url, headers=scrape.headers)
response.status_code

200

In [70]:
soup = BeautifulSoup(response.content, 'html.parser')
soup.prettify()

'<!DOCTYPE html>\n<html class="no-js" lang="nl" xmlns="http://www.w3.org/1999/xhtml">\n <head>\n  <meta charset="utf-8"/>\n  <meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0, maximum-scale=1.0, user-scalable=no" name="viewport">\n   <meta content="IE=edge" http-equiv="X-UA-Compatible"/>\n   <meta content="telephone=no" name="format-detection"/>\n   <title>\n    Ittmann Tandartsen - Velp\n   </title>\n   <link href="https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-ittmann-tandartsen-velp-3043194" rel="canonical"/>\n   <meta content="Bekijk de informatie en lees de waarderingen over Ittmann Tandartsen in Velp. Er zijn 30 waarderingen met een gemiddeld cijfer van 9,8 geplaatst." name="description">\n    <meta content="Tandartsenpraktijk" name="zkn:organisatiesoort"/>\n    <meta content="Ittmann Tandartsen - Velp" property="og:title">\n     <meta content="business.business" property="og:type">\n      <meta content="https://www.zorgkaartnederland.nl

In [71]:
script = soup.find('script', type='application/ld+json')
print(script.prettify())

<script type="application/ld+json">
 {"url":"https:\/\/www.zorgkaartnederland.nl\/zorginstelling\/tandartsenpraktijk-ittmann-tandartsen-velp-3043194","@type":"Organization","name":"Ittmann Tandartsen","telephone":"026-3610403","@context":"https:\/\/schema.org","address":{"@type":"PostalAddress","addressLocality":"Velp","postalCode":"6881 VM","streetAddress":"President Kennedylaan 1 A"},"review":{"@type":"Review","reviewRating":{"@type":"Rating","ratingValue":9.7,"bestRating":10,"worstRating":1},"author":{"@type":"Person","name":"Goedgekeurd door de redactie"},"reviewBody":"Helemaal perfect, je krijgt persoonlijke aandacht. Ik ga met plezier naar de tandarts."},"aggregateRating":{"@type":"AggregateRating","ratingValue":9.8,"ratingCount":30,"bestRating":10,"worstRating":1}}
</script>



In [72]:
data = json.loads(script.string)
data

{'url': 'https://www.zorgkaartnederland.nl/zorginstelling/tandartsenpraktijk-ittmann-tandartsen-velp-3043194',
 '@type': 'Organization',
 'name': 'Ittmann Tandartsen',
 'telephone': '026-3610403',
 '@context': 'https://schema.org',
 'address': {'@type': 'PostalAddress',
  'addressLocality': 'Velp',
  'postalCode': '6881 VM',
  'streetAddress': 'President Kennedylaan 1 A'},
 'review': {'@type': 'Review',
  'reviewRating': {'@type': 'Rating',
   'ratingValue': 9.7,
   'bestRating': 10,
   'worstRating': 1},
  'author': {'@type': 'Person', 'name': 'Goedgekeurd door de redactie'},
  'reviewBody': 'Helemaal perfect, je krijgt persoonlijke aandacht. Ik ga met plezier naar de tandarts.'},
 'aggregateRating': {'@type': 'AggregateRating',
  'ratingValue': 9.8,
  'ratingCount': 30,
  'bestRating': 10,
  'worstRating': 1}}

In [73]:
address = data.get('address')
address

{'@type': 'PostalAddress',
 'addressLocality': 'Velp',
 'postalCode': '6881 VM',
 'streetAddress': 'President Kennedylaan 1 A'}

In [74]:
address.get('addressLocality')

'Velp'

In [75]:
address.get('postalCode')

'6881 VM'

In [76]:
street_address = address.get('streetAddress')
street_address

'President Kennedylaan 1 A'

In [66]:
def split_address(full_address: str) -> Tuple[str, Optional[str], Optional[str]]:
    match = re.match(r'^(.+?)\s(\d+)\s?([a-zA-Z]*)$', full_address.strip())
    if match:
        street: str = match.group(1)
        number: str = match.group(2)
        addition: Optional[str] = match.group(3) if match.group(3) else None
        return street, number, addition
    else:
        return full_address, None, None

In [77]:
split_address(full_address=street_address)

('President Kennedylaan', '1', 'A')